<a href="https://colab.research.google.com/github/david-levin11/meteorology-portfolio/blob/main/Model_TimeSeries_Improved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Model Time Series**
<br/>
This script will attempt to build a time series from various models/blends for the selected variable and then compare it to the observed values at a point over a selected time range.

- David Levin, Arctic Testbed & Proving Ground, Anchorage Alaska

##**1 - Install and Import Packages**
This will take about a minute to run.  You should only have to run this cell one time unless you close or restart this notebook.

In [ ]:
# @title

!pip install eccodes==2.38.3 # use this version to prevent colab from crashing
!pip install herbie-data[extras]
!pip install requests
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from matplotlib.patches import Patch
import numpy as np
import pandas as pd
import xarray as xr

import warnings

import herbie
from herbie import Herbie, FastHerbie

from datetime import datetime, timedelta
import requests
import json
import os

##**2 - Select Your Desired Inputs and Run!**
You can run this cell as many times as you want after you've imported and installed using step 1 above.

In [ ]:

############################ Config & Markdown ###############################
#@markdown What site are we doing a time series for (enter station id such as "pajn")
site = "pajn" #@param {type:"string"}
#@markdown Enter your Synoptic API token below:
token = ""#@param {type:"string"}
#@markdown Which model/blend are we plotting?
model = "nbm" #@param ['hrrrak', 'nbm', 'NamNest', 'RRFS', 'EPS']
#@markdown What time step would you like?
timestep = 3 #@param {type:"integer"}
#runlength = 48
#@markdown Choose Model Run Date
rundate = "2025-08-27" #@param {type:"date"}
#@markdown Choose Model Run Hour (UTC)
hour = 0 #@param {type:"slider", min:0, max:21, step:3}

#@markdown Choose your forecast start hour
start_hour = 0 #@param {type:"slider", min:0, max:240, step:1}

#@markdown Choose your forecast end hour
end_hour = 72 #@param {type:"slider", min:0, max:240, step:1}

#@markdown Choose your weather element to plot (current choices are: **6hrPrecip, 2mTemp,  10mWind, TotalPrecip**)
#@markdown If you select TotalPrecip, you will get the accumulated precipitation between your end and start times. Note that both time steps have to exist in the archive for this to happen.
element = "2mTemp" #@param ['2mTemp', '10mWind', '6hrPrecip', 'TotalPrecip']

############################ Time Logic ##################################

def adjust_to_nearest_divisible_by_3(number):
    if number % 3 == 0:
        return number
    else:
        # Calculate the remainder when the number is divided by 3
        remainder = number % 3
        # Calculate the nearest number divisible by 3
        if remainder == 1:
            return number - 1
        elif remainder == 2:
            return number + 1

def adjust_to_nearest_divisible_by(number, inc):
    # Calculate the remainder when the number is divided by the increment
    remainder = number % inc

    # Calculate the nearest lower and upper multiples of the increment
    lower_multiple = number - remainder
    upper_multiple = lower_multiple + inc

    # Determine which multiple is closer to the original number
    if (number - lower_multiple) < (upper_multiple - number):
        return lower_multiple
    else:
        return upper_multiple

def adjust_initial_hour(hour, date):
    # Define the model run hours including the wrap-around to the next day's 00 hour
    rundate = datetime.strptime(date, "%Y-%m-%d")
    model_hours = [0, 6, 12, 18, 24]  # Adding 24 to represent the next day's 00 hour

    # Find the nearest model hour using the min function with a custom key
    nearest_hour = min(model_hours, key=lambda x: abs(x - hour))

    # Adjust the result if it's 24, convert it back to 0 (00 of the next day)
    if nearest_hour == 24:
        nearest_hour = 0
        newdate = rundate + timedelta(days=1)
        newdate_string = newdate.strftime("%Y-%m-%d")
    else:
        newdate_string = date

    return nearest_hour, newdate_string

# logic for NBM QMD
if model == 'nbm' and element in ["6hrPrecip", "TotalPrecip"] and hour not in [0,6,12,18]:
  print("NBM only produces probabilistic QPF forecasts for the 00, 06, 12, and 18Z runs.  Adjusting your times to reflect this...")
  hour, rundate = adjust_initial_hour(hour,rundate)
  print(f"Your new date is: {rundate}")
  print(f"Your new initialization hour is: {hour}")


if model == 'EPS' and element in ["6hrPrecip", "TotalPrecip"] and hour not in [0,6,12,18]:
  print("NBM only produces probabilistic QPF forecasts for the 00, 06, 12, and 18Z runs.  Adjusting your times to reflect this...")
  hour, rundate = adjust_initial_hour(hour,rundate)
  print(f"Your new date is: {rundate}")
  print(f"Your new initialization hour is: {hour}")

#Getting our model initialization times based on inputs
runhour = str(hour).zfill(2)
runstring = rundate + " " +runhour + ":00"
run_time = rundate + " " +runhour + "Z"

# logic for smaller model runs
if model == 'hrrrak' and start_hour > 48:
  print(f'{model} only runs out to 48 hours!  Adjusting your start time to reflect this!')
  start_hour = 48

# logic for smaller model runs
if model == 'hrrrak' and end_hour > 48:
  print(f'{model} only runs out to 48 hours!  Adjusting your end time to reflect this!')
  end_hour = 48

if model == 'NamNest' or model == 'RRFS':
  if start_hour > 60:
    print(f'{model} only runs out to 60 hours!  Adjusting your start time to reflect this!')
    start_hour = 60

# logic for smaller model runs
if model == 'NamNest' or model == 'RRFS':
  if end_hour > 60:
    print(f'{model} only runs out to 60 hours!  Adjusting your end time to reflect this!')
    end_hour = 60

# Ensure end_hour is greater than start_hour
if end_hour < start_hour:
    end_hour = start_hour

#Adjusting the time step to every 3 hours for Hires models
if model == 'hrrrak' or model == 'NamNest' or model == 'RRFS':
  if timestep % 3 != 0:
    print(f'{model} only has 3 hourly time steps!  Adjusting your timestep to reflect this!')
    timestep = 3


# Adjusting start and end hours to 3 hr increments for Hires models
if model == 'hrrrak' or model == 'NamNest' or model == 'ARW':
  start_hour = adjust_to_nearest_divisible_by(start_hour,timestep)
  end_hour = adjust_to_nearest_divisible_by(end_hour,timestep)

# if we have 6hr precipitation we need 6 hour time steps
if element == "6hrPrecip" or element == "TotalPrecip":
  timestep = 6
  start_hour = adjust_to_nearest_divisible_by(start_hour,timestep)
  end_hour = adjust_to_nearest_divisible_by(end_hour,timestep)
  print("You have selected 6 hr precip but your timestep doesn't match.  Will adjust this for you...")
print(f'Start hour is: {start_hour}')
print(f'End hour is: {end_hour}')
print(f'Timestep is: {timestep}')

# Time range for observations
mw_start = (datetime.strptime(rundate, "%Y-%m-%d") + timedelta(hours=hour) + timedelta(hours=start_hour)).strftime("%Y%m%d%H%M")
mw_end = (datetime.strptime(rundate, "%Y-%m-%d") + timedelta(hours=hour) + timedelta(hours=end_hour)).strftime("%Y%m%d%H%M")

################################# End Time Logic ##############################

product = "sfc"

# Generate the list of forecast hours from start to end with our chosen time step
# Because of the way we calculated precipitation intevals for the EPS have to do things a little differently
if model == "EPS" and element == "6hrPrecip" and start_hour >= 6:
  forecast_hours = list(range(start_hour-6, end_hour + 1, timestep))
  print(f"Selected forecast hour range: {start_hour} to {end_hour}")
  print(f"Forecast hours list: {forecast_hours}")
else:
  forecast_hours = list(range(start_hour, end_hour + 1, timestep))
  print(f"Selected forecast hour range: {start_hour} to {end_hour}")
  print(f"Forecast hours list: {forecast_hours}")

#@markdown Your files will save to the working directory.  To find the
#@markdown file repo by clicking on the folder icon on the left toolbar.
########################## Functions #######################################
def KtoF(temp):
  return (temp - 273.15) * 9/5 + 32

def MMtoIN(precip):
  return precip * 0.0393701

def MtoIN(precip):
  return precip * 39.3701

# Function to fetch data from the Synoptic API
def fetch_synoptic_data(station, start, end, var, unit, api_key, metaronly='0',precip=0):
    base_url = 'https://api.synopticdata.com/v2/stations/timeseries'
    params = {
        'start': start,
        'end': end,
        'obtimezone': 'utc',
        'vars': var,
        'stids': station,
        'units': unit,
        'hfmetars': metaronly,
        'precip': precip,
        'token': api_key
    }
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # Raises HTTPError for bad responses
    return response.json()

# Function to fetch data from the Synoptic API
def fetch_synoptic_data_precip(station, start, end, pmode, interval, unit, api_key, interval_window='0'):
    base_url = 'https://api.synopticdata.com/v2/stations/precipitation'
    params = {
        'start': start,
        'end': end,
        'obtimezone': 'utc',
        'pmode': pmode,
        'interval': interval,
        'units': unit,
        'stids': station,
        'token': api_key
    }
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # Raises HTTPError for bad responses
    return response.json()

def get_metadata(site, token):
  metadata_api = f"https://api.synopticdata.com/v2/stations/metadata?"
  meta_api_args = {"token":token,"stid":site}
  req = requests.get(metadata_api, params=meta_api_args)
  metadata = req.json()
  lat = metadata['STATION'][0]['LATITUDE']
  lon = metadata['STATION'][0]['LONGITUDE']
  site_df = pd.DataFrame({'latitude':round(float(lat),2), 'longitude':round(float(lon),2), 'stid':site.lower()}, index=[0])
  return site_df

# Get the current x-axis limits to dynamically adjust tick spacing
def prune_ticks(ax, max_ticks=4):
    # Get tick locations
    ticks = ax.get_xticks()
    #print(f'Ticks are: {ticks}')
    if len(ticks) > max_ticks:
        # Thin the ticks by keeping every Nth tick
        keep_every = len(ticks) // max_ticks
        #print(f'We are keeping every {keep_every} tick')
        ax.set_xticks(ticks[::keep_every])
        #print(f'New ticks are: {ax.get_xticks()}')

###################### Metadata #############################################

# getting our lat/lon
meta_df = get_metadata(site, token)
#print(meta_df)

############################ Grabbing Model Data #######################################
# accessing our model data based on our inputs
'''
Add in logic to create gust plots for models which have gusts (HRRR, NBM, NamNest?)
Add in logic to pull in wind gust observations
Arrows for wind directions?  Is that possible?
'''
# Extracting model data for the time steps selected
point_vals = []
point_vals_2 = []
point_vals_3 = []
point_vals_4 = []
point_vals_5 = []
point_vals_6 = []
# levels for NBM QMD
nbm_percentiles = [10, 25, 50, 75, 90]
perc_dict = {10: point_vals, 25: point_vals_2, 50: point_vals_3, 75: point_vals_4, 90: point_vals_5}
# for other ensemble systems
valid_times = []
ens_dict ={10:[], 25:[], 50:[], 75:[], 90:[]}
# getting the forecast hours
#print(forecast_hours)
for fcst_hr in forecast_hours:
  valid_time = (datetime.strptime(runstring, "%Y-%m-%d %H:%M") + timedelta(hours=fcst_hr)).strftime("%Y-%m-%d %HZ")
  try:
    # logic for selecting variables
    if element == "10mWind":
      print(f"Now working on {model} {element} for valid time of {valid_time}")
      if model == 'nbm':
        H = Herbie(runstring, model=model, product='ak', fxx=fcst_hr)
        H2 = Herbie(runstring, model=model, product='ak', fxx=fcst_hr)
        H3 = Herbie(runstring, model=model, product='ak', fxx=fcst_hr)
        # 10m wind speed
        ds = H.xarray(f":WIND:10 m above ground:{fcst_hr} hour fcst:nan")
        ds2 = H2.xarray("WDIR:10 m")
        # 10m wind std dev
        ds3 = H3.xarray(f":WIND:10 m above ground:{fcst_hr} hour fcst:ens")
        point_vals.append(ds.herbie.pick_points(meta_df))
        point_vals_2.append(ds2.herbie.pick_points(meta_df))
        point_vals_3.append(ds3.herbie.pick_points(meta_df))
      elif model == 'hrrrak':
        H = Herbie(runstring, model=model, product=product, fxx=fcst_hr)
        ds = H.xarray("[UV]GRD:10 m")
        point_vals.append(ds.herbie.pick_points(meta_df))
      elif model == 'NamNest':
        H = Herbie(runstring, model='nam', product='alaskanest.hiresf', fxx=fcst_hr)
        ds = H.xarray("[UV]GRD:10 m above")
        point_vals.append(ds.herbie.pick_points(meta_df))
      elif model == 'RRFS':
        H = Herbie(runstring, model="rrfs", member='control', domain="alaska", fxx=fcst_hr)
        ds = H.xarray("[UV]GRD:10 m above")
        point_vals.append(ds.herbie.pick_points(meta_df))
      name = '10m Wind Speed'
      description = ds.description

    elif element == "2mTemp":
      print(f"Now working on {model} {element} for valid time of {valid_time}")
      if model == 'nbm':
        H = Herbie(runstring, model=model, product='ak', fxx=fcst_hr)
        ds = H.xarray(f":TMP:2 m above ground:{fcst_hr} hour fcst:nan")
        H2 = Herbie(runstring, model=model, product='ak', fxx=fcst_hr)
        #print(f"DS is {ds}")
        #print(f'Metadata is {meta_df}')
        ds2 = H.xarray(f":TMP:2 m above ground:{fcst_hr} hour fcst:ens")
        point_vals.append(ds.herbie.pick_points(meta_df))
        point_vals_2.append(ds2.herbie.pick_points(meta_df))
      elif model == 'hrrrak':
        H = Herbie(runstring, model=model, product=product, fxx=fcst_hr)
        ds = H.xarray(":TMP:2 m above")
        point_vals.append(ds.herbie.pick_points(meta_df))
      elif model == 'NamNest':
        H = Herbie(runstring, model='nam', product='alaskanest.hiresf', fxx=fcst_hr)
        ds = H.xarray(":TMP:2 m above")
        point_vals.append(ds.herbie.pick_points(meta_df))
      elif model == 'RRFS':
        H = Herbie(runstring, model="rrfs", member='control', domain="alaska", fxx=fcst_hr)
        ds = H.xarray(":TMP:2 m above")
        point_vals.append(ds.herbie.pick_points(meta_df))
      elif model == 'EPS':
        # getting the ensemble forecasts for 2m temperature
        H = Herbie(runstring, model='ifs', product='enfo', fxx=fcst_hr)
        ds = H.xarray(":2t:")
        point_data = ds[0].herbie.pick_points(meta_df)
        valid_time = point_data['valid_time'].values
        temps = point_data['t2m']
        # appending to our lists
        valid_times.append(valid_time)
        for percentile in ens_dict:
          perc_value = KtoF(np.percentile(temps, percentile, axis=0))
          # getting the data out of numpy arrays
          if isinstance(perc_value, np.ndarray) and perc_value.size == 1:
            perc_value = perc_value.item()
          ens_dict[percentile].append(perc_value)
      name = '2m Temperature'
      description = ds.description


    elif element == "6hrPrecip":
      print(f"Now working on {model} {element} for valid time of {valid_time}")
      try:
        #have to subtract the previous total precip accum from the current
        #This won't work unless we have both time steps
        # getting the correct run based on inputs
        if model == "hrrrak":
          H = Herbie(runstring, model=model, product=product, fxx=fcst_hr)
          H2 = Herbie(runstring, model=model, product=product, fxx=fcst_hr-6)
          ds = H.xarray(":APCP:surface:0-[1-9]*")
          ds2 = H2.xarray(":APCP:surface:0-[1-9]*")
          late = ds.herbie.pick_points(meta_df)/25.4
          early = ds2.herbie.pick_points(meta_df)/25.4
          point_vals.append(late-early)
          point_vals_2.append(late)
        elif model == "NamNest":
          H = Herbie(runstring, model='nam', product='alaskanest.hiresf', fxx=fcst_hr)
          H2 = Herbie(runstring, model='nam', product='alaskanest.hiresf', fxx=fcst_hr-3)
          ds = H.xarray(":APCP:")
          ds2 = H2.xarray(":APCP:")
          now = ds.herbie.pick_points(meta_df)/25.4
          prev = ds2.herbie.pick_points(meta_df)/25.4
          point_vals.append(now+prev)
          point_vals_2.append(now)
        elif model == "RRFS":
          H = Herbie(runstring, model="rrfs", member='control', domain="alaska", fxx=fcst_hr)
          H2 = Herbie(runstring, model="rrfs", member='control', domain="alaska", fxx=fcst_hr-6)
          ds = H.xarray(":APCP:surface:0-[1-9]*")
          ds2 = H2.xarray(":APCP:surface:0-[1-9]*")
          late = ds.herbie.pick_points(meta_df)/25.4
          early = ds2.herbie.pick_points(meta_df)/25.4
          point_vals.append(late-early)
          point_vals_2.append(late)
        elif model == "nbm":
          for perc in nbm_percentiles:
            search_string = f":APCP:surface:{fcst_hr-6}-{fcst_hr} hour acc fcst:{perc}% level:nan"
            H = Herbie(runstring, model="nbmqmd", product='ak', fxx=fcst_hr)
            ds = H.xarray(search_string)
            perc_dict[perc].append(ds.herbie.pick_points(meta_df))
          # adding in deterministic NBM
          search_string = f":APCP:surface:{fcst_hr-6}-{fcst_hr} hour acc fcst:nan"
          H = Herbie(runstring, model="nbm", product='ak', fxx=fcst_hr)
          ds = H.xarray(search_string)
          point_vals_6.append(ds.herbie.pick_points(meta_df))
        elif model == "EPS":
          # getting the ensemble forecasts for total precip
          H = Herbie(runstring, model='ifs', product='enfo', fxx=fcst_hr)
          ds = H.xarray(":tp:")
          point_data = ds[0].herbie.pick_points(meta_df)
          #print(point_data)
          valid_time = point_data['valid_time'].values
          precip = point_data['tp']
          # appending to our lists
          valid_times.append(valid_time)
          for percentile in ens_dict:
            perc_value = MtoIN(np.percentile(precip, percentile, axis=0))
            # getting the data out of numpy arrays
            if isinstance(perc_value, np.ndarray) and perc_value.size == 1:
              perc_value = perc_value.item()
            ens_dict[percentile].append(perc_value)
      except Exception as e:
        print(e)
        continue
      name = '6hr Precip'
      description = ds.description
    elif element == "TotalPrecip":
      try:
        print(f"Now working on {model} {element} for valid time of {valid_time}")
        if model == 'hrrrak':
          H = Herbie(runstring, model=model, product=product, fxx=fcst_hr)
          H2 = Herbie(runstring, model=model, product=product, fxx=fcst_hr-3)
          ds = H.xarray(":APCP:surface:0-[1-9]*")
          ds2 = H2.xarray(":APCP:surface:0-[1-9]*")
          late = ds.herbie.pick_points(meta_df)/25.4
          early = ds2.herbie.pick_points(meta_df)/25.4
          point_vals.append(late-early)
          point_vals_2.append(late)
        elif model == "NamNest":
          H = Herbie(runstring, model='nam', product='alaskanest.hiresf', fxx=fcst_hr)
          ds = H.xarray(":APCP:")
          times = ds.herbie.pick_points(meta_df)
          now = ds.herbie.pick_points(meta_df)/25.4
          point_vals.append(now)
          point_vals_2.append(times)
        elif model == "RRFS":
          H = Herbie(runstring, model="rrfs", member='control', domain="alaska", fxx=fcst_hr)
          H2 = Herbie(runstring, model="rrfs", member='control', domain="alaska", fxx=fcst_hr-3)
          ds = H.xarray(":APCP:surface:0-[1-9]*")
          ds2 = H2.xarray(":APCP:surface:0-[1-9]*")
          late = ds.herbie.pick_points(meta_df)/25.4
          early = ds2.herbie.pick_points(meta_df)/25.4
          point_vals.append(late-early)
          point_vals_2.append(late)
        elif model == "nbm":
          for perc in nbm_percentiles:
            search_string = f":APCP:surface:{fcst_hr-6}-{fcst_hr} hour acc fcst:{perc}% level:nan"
            H = Herbie(runstring, model="nbmqmd", product='ak', fxx=fcst_hr)
            ds = H.xarray(search_string)
            perc_dict[perc].append(ds.herbie.pick_points(meta_df))
          # adding in deterministic NBM
          search_string = f":APCP:surface:{fcst_hr-6}-{fcst_hr} hour acc fcst:nan"
          H = Herbie(runstring, model="nbm", product='ak', fxx=fcst_hr)
          ds = H.xarray(search_string)
          point_vals_6.append(ds.herbie.pick_points(meta_df))
        elif model == "EPS":
          # getting the ensemble forecasts for total precip
          H = Herbie(runstring, model='ifs', product='enfo', fxx=fcst_hr)
          ds = H.xarray(":tp:")
          point_data = ds[0].herbie.pick_points(meta_df)
          #print(point_data)
          valid_time = point_data['valid_time'].values
          precip = point_data['tp']
          # appending to our lists
          valid_times.append(valid_time)
          for percentile in ens_dict:
            perc_value = MtoIN(np.percentile(precip, percentile, axis=0))
            # getting the data out of numpy arrays
            if isinstance(perc_value, np.ndarray) and perc_value.size == 1:
              perc_value = perc_value.item()
            ens_dict[percentile].append(perc_value)
      except Exception as e:
        print(e)
        continue
      name = 'Total Precip'
      description = ds.description
  except Exception as e:
    print(e)
    continue

# different logic for EPS 6 hour precip
if model == "EPS" and element == "6hrPrecip":
  # Initialize an empty dictionary to store the 6-hour differences
  new_valid_times = []
  precip_diff = {10: [], 25: [], 50: [], 75: [], 90: []}
  # Loop through valid times and calculate differences only for 6-hour intervals
  for i in range(1, len(valid_times)):
      # Calculate the time difference in hours
      time_diff_hours = (valid_times[i] - valid_times[i-1]) / np.timedelta64(1, 'h')

      if time_diff_hours == 6:
          new_valid_times.append(valid_times[i])
          # If the time difference is exactly 6 hours, calculate precipitation differences
          for percentile in precip_diff:
              diff = ens_dict[percentile][i] - ens_dict[percentile][i-1]
              precip_diff[percentile].append(diff)
      else:
          print(f"Missing some model data for time step {valid_times[i]}")
          print("Skipping this time step")
  valid_times = new_valid_times
  ens_dict = precip_diff
  print(f"Valid times are: {valid_times}")
  print(f"Percentiles are {ens_dict}")

############################# Observed Data ####################################

# now grabbing observations
if element == "2mTemp":
  mw_var = 'air_temp'
  mw_unit = 'temp|F'
  print(f"Now working on observations for {element} from {mw_start} to {mw_end}")
  try:
    synoptic_data = fetch_synoptic_data(site, mw_start, mw_end, mw_var, mw_unit, token)
  except Exception as http_err:
    print(f'Uhoh! It looks like there is something wrong.  Did you forget your Synoptic token?')
    print(http_err)
  #print(synoptic_data)
  # Extract temperature data
  print(f"Success!  Obs found for {element}")
  obs_dates = pd.to_datetime(synoptic_data['STATION'][0]['OBSERVATIONS']['date_time'])
  obs = synoptic_data['STATION'][0]['OBSERVATIONS']['air_temp_set_1']
elif element == "10mWind":
  mw_var = 'wind_speed'
  mw_unit = 'speed|mph'
  print(f"Now working on observations for {element} from {mw_start} to {mw_end}")
  try:
    synoptic_data = fetch_synoptic_data(site, mw_start, mw_end, mw_var, mw_unit, token)
  except Exception as http_err:
    print(f'Uhoh! It looks like there is something wrong.  Did you forget your Synoptic token?')
    print(http_err)
  # Extract wind data
  print(f"Successful API call! Checking for valid obs...")
  #print(synoptic_data)
  try:
    obs_dates = pd.to_datetime(synoptic_data['STATION'][0]['OBSERVATIONS']['date_time'])
    obs = synoptic_data['STATION'][0]['OBSERVATIONS']['wind_speed_set_1']
    print(f"Success!  Obs found for {element}")
  except KeyError:
    print('Uhoh something is wrong with getting your observations!  Check response message below')
    print(synoptic_data['SUMMARY']['RESPONSE_MESSAGE'])
elif element == '6hrPrecip':
  raw_dates = []
  raw_obs = []
  pmode = 'intervals'
  interval = '6'
  mw_unit = 'precip|in'
  print(f"Now working on observations for {element} from {mw_start} to {mw_end}")
  if forecast_hours[0] >= 6:
    # going back 6 extra hours to make sure we grab the 6hr precip at the start of the time range
    mw_start_dt = datetime.strptime(mw_start, "%Y%m%d%H%M") - timedelta(hours=6)
    new_mw_start = mw_start_dt.strftime("%Y%m%d%H%M")
  else:
    new_mw_start = mw_start
  try:
    synoptic_data = fetch_synoptic_data_precip(site, new_mw_start, mw_end, pmode, interval, mw_unit, token)
    #print(synoptic_data)
  except Exception as http_err:
    print(f'Uhoh! It looks like there is something wrong.  Did you forget your Synoptic token?')
    print(http_err)
  # Extract wind data
  print(f"Successful API call! Checking for valid obs...")
  #print(synoptic_data)
  try:
    for total in synoptic_data['STATION'][0]['OBSERVATIONS']['precipitation']:
      raw_dates.append(total['last_report'])
      raw_obs.append(round(float(total['total']),2))
    obs_dates = pd.to_datetime(raw_dates)
    obs = raw_obs
    print(f"Success!  Obs found for {element}")
  except KeyError:
    print('Uhoh something is wrong with getting your observations!  Check response message below')
    print(synoptic_data['SUMMARY']['RESPONSE_MESSAGE'])
elif element == 'TotalPrecip':
  raw_dates = []
  raw_obs = []
  pmode = 'intervals'
  interval = '3'
  mw_unit = 'precip|in'
  if forecast_hours[0] >= 3:
    # going back 3 extra hours to make sure we grab the 3hr precip at the start of the time range
    mw_start_dt = datetime.strptime(mw_start, "%Y%m%d%H%M") - timedelta(hours=6)
    new_mw_start = mw_start_dt.strftime("%Y%m%d%H%M")
  else:
    new_mw_start = mw_start
  print(f"Now working on observations for {element} from {mw_start} to {mw_end}")
  try:
    synoptic_data = fetch_synoptic_data_precip(site, new_mw_start, mw_end, pmode, interval, mw_unit, token)
  except Exception as http_err:
    print(f'Uhoh! It looks like there is something wrong.  Did you forget your Synoptic token?')
    print(http_err)
  # Extract precip accumulation data
  print(f"Successful API call! Checking for valid obs...")
  try:
    for total in synoptic_data['STATION'][0]['OBSERVATIONS']['precipitation']:
      raw_dates.append(total['last_report'])
      raw_obs.append(round(float(total['total']),2))
    obs_dates = pd.to_datetime(raw_dates)
    obs = raw_obs
    print(f"Success!  Obs found for {element}")
  except KeyError:
    print('Uhoh something is wrong with getting your observations!  Check response message below')
    print(synoptic_data['SUMMARY']['RESPONSE_MESSAGE'])
#print(obs_dates)
#print(obs)

########################### Plotting #########################################

# For spacing x-ticks
date_lim = {'nbm':[15,25], 'hrrrak':[10,20], 'NamNest':[10,20], 'RRFS':[10,20]}

if model not in ['EPS']:
  #now plotting our point data
  ts = xr.concat(point_vals, dim="valid_time")
  if point_vals_2 != []:
    ts2 = xr.concat(point_vals_2, dim="valid_time")
    #print(ts2)
  if point_vals_3 != []:
    ts3 = xr.concat(point_vals_3, dim="valid_time")
    #print(ts3)
  if point_vals_4 != []:
    ts4 = xr.concat(point_vals_4, dim="valid_time")
    #print(ts4)
  if point_vals_5 != []:
    ts5 = xr.concat(point_vals_5, dim="valid_time")
    #print(ts5)
  if point_vals_6 != []:
    ts6 = xr.concat(point_vals_6, dim="valid_time")
    #print(ts6)

# for ensemble data
if model == 'nbm' and element in ["6hrPrecip", "TotalPrecip"]:
  ts_dict = {10:ts, 25:ts2, 50:ts3, 75:ts4, 90:ts5}

if model == "EPS":
  # converting any lists to numpy arrays for plotting
  for perc in ens_dict:
    ens_dict[perc] = np.array(ens_dict[perc])

# Set the plot style
plt.style.use('dark_background')
# Create the figure and axis
fig, ax = plt.subplots(figsize=(12, 4))
if element == "2mTemp":
  if model == 'EPS':
    dates = pd.to_datetime(valid_times)
    perc_lines = {10: ["lightcoral", "--" ], 25: ["red", "-"], 50: ["white", "-"], 75: ["red", "-"], 90: ["lightcoral", "--"]}
    for percentile in ens_dict:
      ax.plot(valid_times, ens_dict[percentile], color=perc_lines[percentile][0], linestyle=perc_lines[percentile][1], label=f"{percentile}th Percentile")
    ax.fill_between(dates, ens_dict[25], ens_dict[75], color='red', alpha=0.3)
    #plotting obs
    ax.plot(obs_dates, obs, 'o', label='Observed Temperatures', color='white', markerfacecolor='white', markeredgecolor='black', zorder=6)
    # Customize the grid
    ax.set_xlim(dates.min()-timedelta(hours=3), dates.max()+timedelta(hours=3))
    ax.grid(color='gray', linestyle='--', linewidth=0.5)
    ax.xaxis.set_major_locator(mdates.HourLocator(byhour=[0,6,12,18]))  # Set major ticks every 3 hours
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%a %d %HUTC'))  # Format x-ticks
    # Call the prune function to dynamically thin the ticks
    prune_ticks(ax)
    # Calculate y-axis limits
    min_temp = min(ens_dict[10].min(), (np.array(obs)).min())
    max_temp = max(ens_dict[90].max(), (np.array(obs)).max())
    y_margin = 1  # Degree margin
    y_min = min_temp - y_margin
    y_max = max_temp + y_margin
    ax.set_ylim(y_min, y_max)
    # Set labels
    ax.set_ylabel('2m Temperature(F)', fontsize=12, color='white')
    # Customize tick parameters
    ax.tick_params(axis='x', colors='white', rotation=0)
    ax.tick_params(axis='y', colors='white')
    # Add a legend
    lines, labels = ax.get_legend_handles_labels()
    ax.legend(lines, labels, loc='upper center',  bbox_to_anchor=(0.5, -0.15), ncol=3)
    ax.set_title(f"{model.upper()}--Run: {run_time} Forecast Hours {forecast_hours[0]} To {forecast_hours[-1]}\n{name.title()} Timeseries For {site.upper()}", fontsize=16, color='white', fontweight='bold', loc="center", pad=5)
    # Remove spines for cleaner look
    ax.spines['top'].set_color('none')
    ax.spines['right'].set_color('none')
    # Adjust plot layout for better appearance
    plt.tight_layout()
    figname = f"{site}_{element}_Timeseries_{model}_{rundate}{runhour}_{forecast_hours[0]}_{forecast_hours[-1]}.png"
    plt.savefig(figname)
    plt.show()
  else:
    # Extract data
    valid_time = ts['valid_time'].values
    temperature_kelvin = ts['t2m'].values.squeeze()
    if model == 'nbm':
      standard_dev = ts2['t2m'].values.squeeze()*9/5
    # Convert temperatures from Kelvin to Fahrenheit
    temperature_fahrenheit = (temperature_kelvin - 273.15) * 9/5 + 32
    # Convert valid_time to pandas datetime for easier plotting
    dates = pd.to_datetime(valid_time)
    # Plot each line with specified styles
    ax.plot(dates, temperature_fahrenheit, label='Temperature', color='red', linewidth=2.5)
    # if we're plotting NBM, then plot the standard deviation
    if model == 'nbm':
      plus_sd = temperature_fahrenheit + standard_dev
      minus_sd = temperature_fahrenheit - standard_dev
      ax.fill_between(dates, minus_sd, plus_sd, color='red', alpha=0.3)
    # Plot the observed temperature
    ax.plot(obs_dates, obs, 'o', label='Observed Temperature', color='white', markerfacecolor='white', markeredgecolor='black')
    # Customize the grid
    ax.grid(color='gray', linestyle='--', linewidth=0.5)
    # Set x-axis ticks to match the model's valid times and display date and time
    # Set x-axis ticks to match the model's valid times and display date and time
    if len(dates) > date_lim[model][1]:
      xint = 12
    elif len(dates) > date_lim[model][0] and len(dates) <=date_lim[model][1]:
      xint = 6
    elif len(dates) <= date_lim[model][0]:
      xint = 3
    ax.xaxis.set_major_locator(mdates.HourLocator(interval=xint))  # Set major ticks every 3 hours
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%a %d %HUTC'))  # Format x-ticks
    prune_ticks(ax)
    # Calculate y-axis limits
    if model == 'nbm':
      min_temp = min(temperature_fahrenheit.min() - standard_dev.min(), (np.array(obs)).min())
      max_temp = max(temperature_fahrenheit.max() + standard_dev.max(), (np.array(obs)).max())
    else:
      min_temp = min(temperature_fahrenheit.min(), (np.array(obs)).min())
      max_temp = max(temperature_fahrenheit.max(), (np.array(obs)).max())
    y_margin = 2  # Degree margin
    y_min = min_temp - y_margin
    y_max = max_temp + y_margin
    ax.set_ylim(y_min, y_max)
    # Set labels
    ax.set_ylabel('Temperature (°F)', fontsize=12, color='white')
    # Customize tick parameters
    ax.tick_params(axis='x', colors='white', rotation=0)
    ax.tick_params(axis='y', colors='white')
    # Add a legend
    lines, labels = ax.get_legend_handles_labels()
    ax.legend(lines, labels, loc='upper center',  bbox_to_anchor=(0.5, -0.15), ncol=3)
    ax.set_title(f"{model.upper()}--Run: {run_time} Forecast Hours {forecast_hours[0]} To {forecast_hours[-1]}\n{name.title()} Timeseries For {site.upper()}", fontsize=16, color='white', fontweight='bold', loc="center", pad=5)
    # Remove spines for cleaner look
    ax.spines['top'].set_color('none')
    ax.spines['right'].set_color('none')
    # Adjust plot layout for better appearance
    plt.tight_layout()
    figname = f"{element}_Timeseries_{model}_{rundate}{runhour}_{forecast_hours[0]}_{forecast_hours[-1]}.png"
    # Display the plot
    plt.savefig(figname)
    plt.show()
elif element == "6hrPrecip":
  # for plotting non-probabilistic models
  if model == "hrrrak" or model == "NamNest" or model == "RRFS":
    # Extract data
    valid_time = ts2['valid_time'].values
    precip = ts['tp'].values.squeeze()
    # Convert valid_time to pandas datetime for easier plotting
    dates = pd.to_datetime(valid_time)
    # Plot each line with specified styles
    bar_width = 1/24
    time_offset = 1/24
    # Plot the observed precip
    dates_num = mdates.date2num(dates)
    obs_dates_num = mdates.date2num(obs_dates)
    fcst_bar = ax.bar(dates_num-time_offset/2, precip, bar_width, label='6hr Precipitation', color='green', edgecolor='white')
    obs_bar = ax.bar(obs_dates_num+time_offset/2, obs, bar_width, label='Observed 6hr Precipitation', color='white', edgecolor='black')
    # Customize the grid
    ax.set_xlim(dates.min()-timedelta(hours=6), dates.max()+timedelta(hours=6))
    ax.grid(color='gray', linestyle='--', linewidth=0.5)
    ax.xaxis.set_major_locator(mdates.HourLocator(byhour=[0,6,12,18]))  # Set major ticks every 3 hours
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%a %d %HUTC'))  # Format x-ticks
    # Call the prune function to dynamically thin the ticks
    prune_ticks(ax)
    # Calculate y-axis limits
    min_temp = min(precip.min(), (np.array(obs)).min())
    max_temp = max(precip.max(), (np.array(obs)).max())
    y_margin = 0.1  # Degree margin
    y_min = -0.01
    y_max = max_temp + y_margin
    ax.set_ylim(y_min, y_max)
    # Set labels
    ax.set_ylabel('6hr Precip (in)', fontsize=12, color='white')
    # Customize tick parameters
    ax.tick_params(axis='x', colors='white', rotation=0)
    ax.tick_params(axis='y', colors='white')
    # Add a legend
    lines, labels = ax.get_legend_handles_labels()
    ax.legend(lines, labels, loc='upper center',  bbox_to_anchor=(0.5, -0.15), ncol=3)
    ax.set_title(f"{model.upper()}--Run: {run_time} Forecast Hours {forecast_hours[0]} To {forecast_hours[-1]}\n{name.title()} Timeseries For {site.upper()}", fontsize=16, color='white', fontweight='bold', loc="center", pad=5)
    # Remove spines for cleaner look
    ax.spines['top'].set_color('none')
    ax.spines['right'].set_color('none')
    # Adjust plot layout for better appearance
    plt.tight_layout()
    figname = f"{site}_{element}_Timeseries_{model}_{rundate}{runhour}_{forecast_hours[0]}_{forecast_hours[-1]}.png"
    plt.savefig(figname)
    plt.show()
  # for plotting probabilistic models we'll use a box and whisker plot
  elif model == "nbm":
    # Extract data
    percentile_data = {}
    valid_time = ts['valid_time'].values
    # populating our percentile lists
    for percentile in ts_dict:
      percentile_data.update({percentile: ts_dict[percentile]["tp"].values.squeeze()/25.4})
    # creating a dataset for our box plot
    # if we only have one time step we get errors:
    if len(valid_time) == 1:
      box_data = [[percentile_data[10], percentile_data[25], percentile_data[50], percentile_data[75], percentile_data[90]]]
    else:
      box_data = []
      for i in range(0,len(valid_time)):
        box_data.append([percentile_data[10][i], percentile_data[25][i], percentile_data[50][i], percentile_data[75][i], percentile_data[90][i]])
    # getting our deterministic dataset
    det_nbm = ts6['tp'].values.squeeze()/25.4
    # Convert valid_time to pandas datetime for easier plotting
    dates = pd.to_datetime(valid_time)
    # Create a box and whisker plot for each time step
    widths= 1/24
    boxprops = dict(facecolor='green', edgecolor='white')
    whiskerprops = dict(color='white', linewidth=2)
    ax.boxplot(box_data, positions=mdates.date2num(dates), widths=widths, patch_artist=True, boxprops=boxprops, whiskerprops=whiskerprops)
    #plotting deterministic
    ax.plot(dates, det_nbm, 'o', label='Deterministic NBM', color='yellow', markerfacecolor='yellow', markeredgecolor='black')
    #plotting obs
    ax.plot(obs_dates, obs, 'o', label='Observed 6hr Precipitation', color='white', markerfacecolor='white', markeredgecolor='black')
    # Customize the grid
    # Add a legend
    # Create a patch for the box and whiskers for the legend
    box_patch = Patch(facecolor='green', edgecolor='white', label='Precipitation Percentiles (10th-90th)')
    det_patch = Patch(facecolor='yellow', edgecolor='black', label='Deterministic NBM')
    obs_patch = Patch(facecolor='white', edgecolor='black', label='Observed 6hr Precipitation')
    lines, labels = ax.get_legend_handles_labels()
    ax.legend(handles=[box_patch, det_patch, obs_patch], loc='upper center',  bbox_to_anchor=(0.5, -0.15), ncol=3)
    ax.set_xlim(dates.min()-timedelta(hours=6), dates.max()+timedelta(hours=6))
    ax.grid(color='gray', linestyle='--', linewidth=0.5)
    ax.xaxis.set_major_locator(mdates.HourLocator(byhour=[0,6,12,18]))  # Set major ticks every 3 hours
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%a %d %HUTC'))  # Format x-ticks
    # Call the prune function to dynamically thin the ticks
    prune_ticks(ax)
    # Calculate y-axis limits
    min_temp = min(percentile_data[10].min(), (np.array(obs)).min())
    max_temp = max(percentile_data[90].max(), (np.array(obs)).max())
    y_margin = 0.1  # Degree margin
    y_min = -0.01
    y_max = max_temp + y_margin
    ax.set_ylim(y_min, y_max)
    # Set labels
    ax.set_ylabel('6hr Precip (in)', fontsize=12, color='white')
    # Customize tick parameters
    ax.tick_params(axis='x', colors='white', rotation=0)
    ax.tick_params(axis='y', colors='white')
    ax.set_title(f"{model.upper()}--Run: {run_time} Forecast Hours {forecast_hours[0]} To {forecast_hours[-1]}\n{name.title()} Timeseries For {site.upper()}", fontsize=16, color='white', fontweight='bold', loc="center", pad=5)
    # Remove spines for cleaner look
    ax.spines['top'].set_color('none')
    ax.spines['right'].set_color('none')
    # Adjust plot layout for better appearance
    plt.tight_layout()
    figname = f"{site}_{element}_Timeseries_{model}_{rundate}{runhour}_{forecast_hours[0]}_{forecast_hours[-1]}.png"
    # Display the plot
    plt.savefig(figname)
    plt.show()
  elif model == "EPS":
    print(f"{element} for {model} is still a work in progress...check back later!  Sorry!")
elif element == "TotalPrecip":
  # for plotting non-probabilistic models
  if model == "hrrrak" or model == "NamNest" or model == "RRFS":
    # Extract data
    valid_time = ts2['valid_time'].values
    # calculating running accumulations
    precip = np.cumsum(ts['tp'].values.squeeze())
    obs = np.cumsum(obs)
    # Convert valid_time to pandas datetime for easier plotting
    dates = pd.to_datetime(valid_time)
    # Plot each line with specified styles
    ax.plot(dates, precip, label='Accumulated Precipitation', color='green', linewidth=2.5)
    # Plot the observed temperature
    ax.plot(obs_dates, obs, 'o', label='Observed Accumulated Precip', color='green', markerfacecolor='green', markeredgecolor='white')
    # Customize the grid
    ax.set_xlim(dates.min()-timedelta(hours=3), dates.max()+timedelta(hours=3))
    ax.grid(color='gray', linestyle='--', linewidth=0.5)
    ax.xaxis.set_major_locator(mdates.HourLocator(byhour=[0,6,12,18]))  # Set major ticks every 3 hours
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%a %d %HUTC'))  # Format x-ticks
    # Call the prune function to dynamically thin the ticks
    prune_ticks(ax)
    # Calculate y-axis limits
    min_temp = min(precip.min(), (np.array(obs)).min())
    max_temp = max(precip.max(), (np.array(obs)).max())
    y_margin = 0.1  # Degree margin
    y_min = -0.01
    y_max = max_temp + y_margin
    ax.set_ylim(y_min, y_max)
    # Set labels
    ax.set_ylabel('6hr Precip (in)', fontsize=12, color='white')
    # Customize tick parameters
    ax.tick_params(axis='x', colors='white', rotation=0)
    ax.tick_params(axis='y', colors='white')
    # Add a legend
    lines, labels = ax.get_legend_handles_labels()
    ax.legend(lines, labels, loc='upper center',  bbox_to_anchor=(0.5, -0.15), ncol=3)
    ax.set_title(f"{model.upper()}--Run: {run_time} Forecast Hours {forecast_hours[0]} To {forecast_hours[-1]}\n{name.title()} Timeseries For {site.upper()}", fontsize=16, color='white', fontweight='bold', loc="center", pad=5)
    # Remove spines for cleaner look
    ax.spines['top'].set_color('none')
    ax.spines['right'].set_color('none')
    # Adjust plot layout for better appearance
    plt.tight_layout()
    figname = f"{site}_{element}_Timeseries_{model}_{rundate}{runhour}_{forecast_hours[0]}_{forecast_hours[-1]}.png"
    plt.savefig(figname)
    plt.show()
  # for plotting probabilistic models we'll use a box and whisker plot
  elif model == "nbm":
    perc_lines = {10: ["lime", "--" ], 25: ["green", "-"], 50: ["white", "-"], 75: ["green", "-"], 90: ["lime", "--"]}
    # Extract data
    obs = np.cumsum(obs)
    det_nbm = np.cumsum(ts6["tp"].values.squeeze()/25.4)
    # Extract data
    percentile_data = {}
    valid_time = ts['valid_time'].values
    # populating our percentile lists
    for percentile in ts_dict:
      percentile_data.update({percentile: np.cumsum(ts_dict[percentile]["tp"].values.squeeze()/25.4)})
    # Convert valid_time to pandas datetime for easier plotting
    dates = pd.to_datetime(valid_time)
    # Plot each line with specified styles
    for percentile in percentile_data:
      ax.plot(dates, percentile_data[percentile], label=f'{percentile}th Percentile', color=perc_lines[percentile][0], linestyle=perc_lines[percentile][1], linewidth=2.5)
    ax.fill_between(dates, percentile_data[25], percentile_data[75], color='green', alpha=0.3)
    #plotting obs
    ax.plot(obs_dates, obs, 'o', label='Observed Accumulated Precip', color='white', markerfacecolor='white', markeredgecolor='black', zorder=6)
    #plotting deterministic
    ax.plot(dates, det_nbm, 'o', label='Deterministic NBM', color='yellow', markerfacecolor='yellow', markeredgecolor='black', zorder=7)
    # Customize the grid
    ax.set_xlim(dates.min()-timedelta(hours=3), dates.max()+timedelta(hours=3))
    ax.grid(color='gray', linestyle='--', linewidth=0.5)
    ax.xaxis.set_major_locator(mdates.HourLocator(byhour=[0,6,12,18]))  # Set major ticks every 3 hours
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%a %d %HUTC'))  # Format x-ticks
    # Call the prune function to dynamically thin the ticks
    prune_ticks(ax)
    # Calculate y-axis limits
    min_temp = min(percentile_data[10].min(), (np.array(obs)).min())
    max_temp = max(percentile_data[90].max(), (np.array(obs)).max())
    y_margin = 0.1  # Degree margin
    y_min = -0.01
    y_max = max_temp + y_margin
    ax.set_ylim(y_min, y_max)
    # Set labels
    ax.set_ylabel('Accumulated Precip (in)', fontsize=12, color='white')
    # Customize tick parameters
    ax.tick_params(axis='x', colors='white', rotation=0)
    ax.tick_params(axis='y', colors='white')
    # Add a legend
    lines, labels = ax.get_legend_handles_labels()
    ax.legend(lines, labels, loc='upper center',  bbox_to_anchor=(0.5, -0.15), ncol=3)
    ax.set_title(f"{model.upper()}--Run: {run_time} Forecast Hours {forecast_hours[0]} To {forecast_hours[-1]}\n{name.title()} Timeseries For {site.upper()}", fontsize=16, color='white', fontweight='bold', loc="center", pad=5)
    # Remove spines for cleaner look
    ax.spines['top'].set_color('none')
    ax.spines['right'].set_color('none')
    # Adjust plot layout for better appearance
    plt.tight_layout()
    figname = f"{site}_{element}_Timeseries_{model}_{rundate}{runhour}_{forecast_hours[0]}_{forecast_hours[-1]}.png"
    plt.savefig(figname)
    plt.show()
  elif model == "EPS":
    perc_lines = {10: ["lime", "--" ], 25: ["green", "-"], 50: ["white", "-"], 75: ["green", "-"], 90: ["lime", "--"]}
    # Extract data
    dates = pd.to_datetime(valid_times)
    for percentile in ens_dict:
      ax.plot(dates, ens_dict[percentile], label=f'{percentile}th Percentile', color=perc_lines[percentile][0], linestyle=perc_lines[percentile][1], linewidth=2.5)
    ax.fill_between(dates, ens_dict[25], ens_dict[75], color='green', alpha=0.3)
    #plotting obs
    obs = np.cumsum(obs)
    ax.plot(obs_dates, obs, 'o', label='Observed Accumulated Precip', color='white', markerfacecolor='white', markeredgecolor='black', zorder=6)
    # Customize the grid
    ax.set_xlim(dates.min()-timedelta(hours=3), dates.max()+timedelta(hours=3))
    ax.grid(color='gray', linestyle='--', linewidth=0.5)
    ax.xaxis.set_major_locator(mdates.HourLocator(byhour=[0,6,12,18]))  # Set major ticks every 3 hours
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%a %d %HUTC'))  # Format x-ticks
    # Call the prune function to dynamically thin the ticks
    prune_ticks(ax)
    # Calculate y-axis limits
    min_temp = min(ens_dict[10].min(), (np.array(obs)).min())
    max_temp = max(ens_dict[90].max(), (np.array(obs)).max())
    y_margin = 0.1  # Degree margin
    y_min = -0.01
    y_max = max_temp + y_margin
    ax.set_ylim(y_min, y_max)
    # Set labels
    ax.set_ylabel('Accumulated Precip (in)', fontsize=12, color='white')
    # Customize tick parameters
    ax.tick_params(axis='x', colors='white', rotation=0)
    ax.tick_params(axis='y', colors='white')
    # Add a legend
    lines, labels = ax.get_legend_handles_labels()
    ax.legend(lines, labels, loc='upper center',  bbox_to_anchor=(0.5, -0.15), ncol=3)
    ax.set_title(f"{model.upper()}--Run: {run_time} Forecast Hours {forecast_hours[0]} To {forecast_hours[-1]}\n{name.title()} Timeseries For {site.upper()}", fontsize=16, color='white', fontweight='bold', loc="center", pad=5)
    # Remove spines for cleaner look
    ax.spines['top'].set_color('none')
    ax.spines['right'].set_color('none')
    # Adjust plot layout for better appearance
    plt.tight_layout()
    figname = f"{site}_{element}_Timeseries_{model}_{rundate}{runhour}_{forecast_hours[0]}_{forecast_hours[-1]}.png"
    plt.savefig(figname)
    plt.show()
elif element == "10mWind":
  # Extract data from most models
  if model == 'hrrrak' or model == 'NamNest' or model == 'RRFS':
    valid_time = ts['valid_time'].values
    uwind = ts['u10'].values.squeeze()
    vwind = ts['v10'].values.squeeze()
    print(valid_time)
    # calculating magnitude and direction
    wind_speed = np.sqrt(uwind**2 + vwind**2)
    wind_direction = (np.degrees(np.arctan2(-uwind, -vwind)) +360) % 360
    #print(wind_direction)
  elif model == 'nbm':
    valid_time = ts['valid_time'].values
    wind_speed = ts['si10'].values.squeeze()
    wind_direction = ts2['wdir10'].values.squeeze()
    standard_dev = ts3['si10'].values.squeeze()*2.2369
  # Convert winds from m/s to mph
  wind_speed_mph = wind_speed * 2.2369
  # Convert valid_time to pandas datetime for easier plotting
  dates = pd.to_datetime(valid_time)
  # Plot each line with specified styles
  ax.plot(dates, wind_speed_mph, label='Wind Speed', color='lime', linewidth=1.5)
  # if we're plotting NBM, then plot the standard deviation
  if model == 'nbm':
    plus_sd = wind_speed_mph + standard_dev
    minus_sd = wind_speed_mph - standard_dev
    minus_sd = np.where(minus_sd >=0, minus_sd, 0)
    ax.fill_between(dates, minus_sd, plus_sd, color='lime', alpha=0.3)
  # Plot the observed temperature
  ax.plot(obs_dates, obs, 'o', label='Observed Wind Speed', color='lime', markerfacecolor='lime', markeredgecolor='white')
  # Customize the grid
  ax.grid(color='gray', linestyle='--', linewidth=0.5)
  # Set x-axis ticks to display date and time
  ax.set_xlim(dates.min(), dates.max())
  # Set x-axis ticks to match the model's valid times and display date and time
  if len(dates) > date_lim[model][1]:
    xint = 12
  elif len(dates) > date_lim[model][0] and len(dates) <=date_lim[model][1]:
    xint = 6
  elif len(dates) <= date_lim[model][0]:
    xint = 3
  ax.set_xlim(dates.min(), dates.max())
  ax.xaxis.set_major_locator(mdates.HourLocator(interval=xint))  # Set major ticks every 3 hours
  ax.xaxis.set_major_formatter(mdates.DateFormatter('%a %d %HUTC'))  # Format x-ticks
  #thinning x-ticks if necessary
  prune_ticks(ax)
  # Calculate y-axis limits
  obsarray = np.array([x if x is not None else np.nan for x in obs])
  if model == 'nbm':
    min_temp = min(wind_speed_mph.min() - standard_dev.min(), (np.nanmin(obsarray)))
    max_temp = max(wind_speed_mph.max() + standard_dev.max(), (np.nanmax(obsarray)))
  else:
    min_temp = min(wind_speed_mph.min(), (np.nanmin(obsarray)))
    max_temp = max(wind_speed_mph.max(), (np.nanmax(obsarray)))
  if element == "10mWind":
    y_minmargin = 2  # Degree margin
    y_maxmargin = 4
  elif element == "2mTemp":
    y_minmargin = 2  # Degree margin
    y_maxmargin = 2
  y_min = min_temp - y_minmargin
  y_max = max_temp + y_maxmargin
  ax.set_ylim(y_min, y_max)
  # Set labels
  ax.set_ylabel('Wind Speed (MPH)', fontsize=12, color='white')
  # Customize tick parameters
  ax.tick_params(axis='x', colors='white', rotation=0)
  ax.tick_params(axis='y', colors='white')
  # Create secondary y-axis for wind direction
  ax2 = ax.twinx()
  ax2.set_ylim(0, 360)
  ax2.set_ylabel('Direction (°)', color='white')
  # Plot wind direction as circles
  ax2.plot(dates, wind_direction, 'o', color='gray', label='10 m Wind Dir')
  # Combine legends from both axes
  lines, labels = ax.get_legend_handles_labels()
  lines2, labels2 = ax2.get_legend_handles_labels()
  ax2.legend(lines + lines2, labels + labels2, loc='upper center',  bbox_to_anchor=(0.5, -0.15), ncol=3)
  # Adjust plot layout for better appearance
  # Remove spines for cleaner look
  ax.spines['top'].set_color('none')
  ax.spines['right'].set_color('none')
  ax2.spines['top'].set_color('none')
  ax2.spines['right'].set_color('none')
  fig.suptitle(f"{model.upper()}--Run: {run_time} Forecast Hours {forecast_hours[0]} To {forecast_hours[-1]}\n10m Wind Timeseries For {site.upper()}", fontsize=16, color='white', fontweight='bold')
  plt.tight_layout()
  figname = f"{site}_{element}_Timeseries_{model}_{rundate}{runhour}_{forecast_hours[0]}_{forecast_hours[-1]}.png"
  plt.savefig(figname)
  plt.show()
